<a href="https://colab.research.google.com/github/AntonioMarsella/website/blob/master/reshaping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


TensorFlow 2.x selected.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [104]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_addons as tfa
import datetime
import tensorflow as tf
import os
from dataloader2 import Dataset
import numpy as np
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
print(tf.__version__)


2.1.0


In [0]:
split_dict = {'IXMAS': { 
    'allocentric': {
        'train':['cam0','cam1','cam2','cam3'],
        'test':['cam4']},
    'lateral': {
        'train':['cam0','cam3'],
        'test':['cam1','cam2','cam4']},
    'simple': {
        'train':['cam0','cam1','cam2'],
        'test':['cam3']},
    'crazy': {
        'train':['cam0'],
        'test':['cam4']}
                        }
             }

In [0]:
dataset = Dataset(batch_size = 4, data_path_index= 1)
dataset.load_dataset_paths()


In [107]:

dataset.crossview_split(split_dict['IXMAS']['allocentric'])


Training samples: 1056
Validation samples: 264
Test samples: 330


In [0]:
num_classes = 11
width, height = 7, 7
num_channels = 1024
input_shape = (8, width, height, num_channels)



In [0]:
@tf.custom_gradient
def GradientReversalOperator(x):
    def grad(dy):
        #tf.print(tf.math.reduce_mean(dy))
        return -1 * dy
    return x, grad

class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(GradientReversalLayer, self).__init__()
        
    def call(self, inputs):
        return GradientReversalOperator(inputs)


In [0]:

import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout, Conv3D
from tensorflow.keras.regularizers import l2

MAX_STEP = 10000

log_format = 'L1 Test: {:.4f}, Acc1 Test: {:.2f}\n'+ \
    'L2 Test: {:.4f}, Acc2 Test: {:.2f}\n'+ \
    'L3 Test: {:.4f}, Acc3 Test: {:.2f}\n'

log_format_source_only = 'L1 Test: {:.4f}, Acc1 Test: {:.2f}\n' + \
    'L1 Target_1: {:.4f}, Acc3 Target_1: {:.2f}\n' + \
    'L1 Target_2: {:.4f}, Acc3 Target_2: {:.2f}\n'


In [0]:
class InvertedLoss(tf.keras.losses.Loss):

    def __init__(self, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE, name='InvertedLoss'):
        super().__init__(reduction=reduction, name=name)
    
    @tf.function 
    def call(self, y_true, y_pred, lp_loss = 1):
        standard_loss = tf.keras.losses.SparseCategoricalCrossentropy()
        #return tf.math.maximum(standard_loss(y_true, y_pred),tf.math.log(tf.divide(tf.constant(1.0, dtype = 'float32'),(standard_loss(y_true, y_pred)))))
        #return tf.math.maximum(standard_loss(y_true, y_pred), tf.multiply(2.0, tf.math.exp(tf.multiply(-1.0, tf.math.square(standard_loss(y_true, y_pred))))))
        return tf.multiply(1.0, tf.math.exp(tf.multiply(-1.0, tf.math.abs(standard_loss(y_true, y_pred)))))

In [0]:
class EuclideanDistanceLoss(tf.keras.losses.Loss):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    def __init__(self, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE, name='EuclideanDistanceLoss'):
        super().__init__(reduction=reduction, name=name)
    def call(self, y_true, y_pred):
      return 0.5 * tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_true - y_pred), axis=-1))

In [0]:

import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling3D, Flatten, Dense, Dropout, Conv3D, UpSampling3D
from tensorflow.keras.regularizers import l2


MAX_STEP = 10000

log_format = 'L1 Test: {:.4f}, Acc1 Test: {:.2f}\n'+ \
    'L2 Test: {:.4f}, Acc2 Test: {:.2f}\n'+ \
    'L3 Test: {:.4f}, Acc3 Test: {:.2f}\n'

log_format_source_only = 'L1 Test: {:.4f}, Acc1 Test: {:.2f}\n' + \
    'L1 Target_1: {:.4f}, Acc3 Target_1: {:.2f}\n' + \
    'L1 Target_2: {:.4f}, Acc3 Target_2: {:.2f}\n'



class Upsample():
    def __init__(self, input_shape = input_shape, run_name = 'multiclass', lr = [0.1, 0.1, 0.1], source_only=False, category = (None, None)):
        super(Upsample, self).__init__()

        self.downsample = Sequential([
                                             
            Conv3D(filters=104, kernel_size=[2,2,5], kernel_regularizer=l2(0.001), padding='same', input_shape=input_shape),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling3D([2,2,2]),
            Conv3D(filters=52,kernel_size=[1,1,3], kernel_regularizer=l2(0.001), padding='same'),
            Activation('relu'),
            Conv3D(filters=26,kernel_size=[1,1,2], kernel_regularizer=l2(0.001), padding='same'),
            Activation('relu')
            ])
        
        self.upsample = Sequential([
            UpSampling3D([2,2,2]), 
            BatchNormalization(),
            Activation('tanh')                    
        ])
        self.label_predictor = Sequential([
            Flatten(),
            #Dense(128, activation='relu'),
            #BatchNormalization(),
            Dropout(0.5),
            Dense(11, activation = None),
            BatchNormalization(),
            Activation('softmax')
        ])
            
        self.feature_extractor = Sequential([
            self.downsample,
            self.upsample                       
        ])

        self.domain_classifier = Sequential([
            GradientReversalLayer(),                                 
            Conv3D(filters=104, kernel_size=[1,1,1], kernel_regularizer=l2(0.001), padding='same'),
            Activation('relu'),
            Flatten(),
            #Dense(128, kernel_regularizer=l2(0.001)),
            #BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(5, activation='softmax'),
                    
            ])
        
       # self.domain_classifier2 = Sequential([
       #     GradientReversalLayer(),
       #     Flatten(),
       #     Dropout(0.5),
       #     Dense(5, activation='softmax')
       # ])
        
        self.predict_label = Sequential([
            self.feature_extractor,
            self.label_predictor
            ])

        self.classify_domain = Sequential([
            self.downsample,
            self.domain_classifier
            ])
        
        #self.classify_domain2 = Sequential([
        #    self.downsample,
        #    self.domain_classifier2
        #    ])
        
        
        self.loss = tf.keras.losses.SparseCategoricalCrossentropy()
        self.inv_loss = InvertedLoss()
        self.lp_lr = lr[0]
        self.dc_lr = lr[1]
        self.fe_lr = lr[2]

        # self.lp_optimizer = tf.keras.optimizers.Adam(learning_rate=lr[0])
        # self.dc_optimizer = tf.keras.optimizers.Adam(learning_rate=lr[1])
        # self.fe_optimizer = tf.keras.optimizers.Adam(learning_rate=lr[2]) #fe_lr=0.0005


        self.lp_optimizer = tf.keras.optimizers.Adam(learning_rate=self.lp_lr, decay=0.0005)
        self.dc_optimizer = tf.keras.optimizers.Adam(learning_rate=self.dc_lr, decay=0.0002)
        self.fe_optimizer = tf.keras.optimizers.Adam(learning_rate=self.fe_lr, decay = 0.0002) #fe_lr=0.0005

        self.train_lp_loss = tf.keras.metrics.Mean()
        self.train_dc_loss = tf.keras.metrics.Mean()
        
        self.train_lp_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        self.train_dc_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

        self.test_lp_loss = tf.keras.metrics.Mean()
        self.test_dc_loss = tf.keras.metrics.Mean()
        self.test_target_lp_loss = tf.keras.metrics.Mean()
        self.test_target_standard_lp_loss = tf.keras.metrics.Mean()
        self.test_lp_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        self.test_dc_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        self.test_target_lp_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        self.test_target_standard_lp_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        self.train_target_lp_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        self.euclidean_loss = EuclideanDistanceLoss()
        if source_only:
            self.target_accuracy_1 = tf.keras.metrics.SparseCategoricalAccuracy()
            self.target_accuracy_2 = tf.keras.metrics.SparseCategoricalAccuracy()
            self.target_loss_1 = tf.keras.metrics.Mean()
            self.target_loss_2 = tf.keras.metrics.Mean()

        self.create_logger(run_name, source_only, category)
    
    @tf.function
    def single_train(self, x_class, y_class, views_class, x_domain, views_domain, minimize_distance = False):
        domain_labels = tf.concat([views_class, views_domain], axis = 0)
        x_both = tf.concat([x_class, x_domain], axis = 0)

        with tf.GradientTape() as tape:
          y_class_pred = self.predict_label(x_class, training=True)
          lp_loss = tf.multiply(2.0, self.loss(y_class, y_class_pred))
          tf.print('lp_loss', lp_loss)
          lp_grad = tape.gradient(lp_loss, self.predict_label.trainable_variables)

        #with tf.GradientTape() as tape:    
        #  y_domain_pred = self.classify_domain(x_both, training=True)
        #  inv_loss = self.inv_loss(domain_labels, y_domain_pred)
        #  tf.print('inv_loss', inv_loss)
        #  fe_grad = tape.gradient(inv_loss, self.classify_domain.trainable_variables)

        with tf.GradientTape(persistent = True) as tape:
          y_domain_pred = self.classify_domain(x_both, training=True)
          dc_loss = self.loss(domain_labels, y_domain_pred)
          
          tf.print('dc_loss', dc_loss)
          dc_grad = tape.gradient(dc_loss, self.domain_classifier.trainable_variables)
          
          fe_inv_grad = tape.gradient(dc_loss, self.classify_domain.trainable_variables)
       
        #with tf.GradientTape(persistent = True) as tape:
        #  y_domain_pred2 = self.classify_domain2(x_both, training=True)
        #  dc_loss2 = self.loss(domain_labels, y_domain_pred2)
          
        #  tf.print('dc_loss2', dc_loss2)
        #  dc_grad2 = tape.gradient(dc_loss2, self.domain_classifier2.trainable_variables)
        #  downsample2_grad = tape.gradient(dc_loss2, self.classify_domain2.trainable_variables)

        self.lp_optimizer.apply_gradients(zip(lp_grad, self.predict_label.trainable_variables))
        #self.fe_optimizer.apply_gradients(zip(fe_grad, self.classify_domain.trainable_variables))
        self.dc_optimizer.apply_gradients(zip(dc_grad, self.domain_classifier.trainable_variables))
        #self.dc_optimizer.apply_gradients(zip(dc_grad2, self.domain_classifier2.trainable_variables))
        #self.dc_optimizer.apply_gradients(zip(dc_grad3, self.domain_classifier_features.trainable_variables))
        #self.fe_optimizer.apply_gradients(zip(fe_grad, self.downsample.trainable_variables))
        #self.fe_optimizer.apply_gradients(zip(downsample2_grad, self.downsample.trainable_variables))
        #self.fe_optimizer.apply_gradients(zip(dc_features_grad, self.feature_extractor.trainable_variables))
        self.fe_optimizer.apply_gradients(zip(fe_inv_grad, self.downsample.trainable_variables))

        if minimize_distance:
            with tf.GradientTape(persistent = True) as tape:
                distance_loss = self.euclidean_loss(self.return_latent_variables(x_class), x_class) 
            distance_grad = tape.gradient(distance_loss, self.feature_extractor.trainable_variables)
            self.fe_optimizer.apply_gradients(zip(distance_grad, self.feature_extractor.trainable_variables))
            tf.print(distance_loss)
        del tape

        #self.train_lp_loss(lp_loss)
        #self.train_lp_accuracy(y_class, y_class_pred)
        
        #self.train_dc_loss(dc_loss)
        #self.train_dc_accuracy(domain_labels, y_domain_pred)

        return

    @tf.function
    def test_target_test(self, x_domain, y_domain):
        
        with tf.GradientTape() as tape:
            y_target_class_pred = self.predict_label(x_domain, training=False)
            target_lp_loss = self.loss(y_domain, y_target_class_pred)
        
        self.test_target_lp_loss(target_lp_loss)
        self.test_target_lp_accuracy(y_domain, y_target_class_pred)

        return
    @tf.function
    def test_target(self, x_domain, y_domain):
        
        with tf.GradientTape() as tape:
            y_target_class_pred = self.predict_label(x_domain, training=False)
            target_lp_loss = self.loss(y_domain, y_target_class_pred)
        
        self.test_target_standard_lp_loss(target_lp_loss)
        self.test_target_standard_lp_accuracy(y_domain, y_target_class_pred)

        return

    @tf.function
    def test_source_only(self, x_domain, y_domain, domain_labels):
        
        with tf.GradientTape() as tape:
            y_target_class_pred = self.predict_label(x_domain, training=False)
            target_lp_loss = self.loss(y_domain, y_target_class_pred)

        with tf.GradientTape(persistent = True) as tape:
              predicted_view = self.classify_domain(x_domain, training=False)
              dc_loss = self.loss(domain_labels, predicted_view)

        self.train_lp_loss(target_lp_loss)
        self.train_lp_accuracy(y_domain, y_target_class_pred)  

        self.train_dc_loss(dc_loss)
        self.train_dc_accuracy(domain_labels, predicted_view)


    @tf.function
    def test_only(self, x_domain, y_domain, mode):
        
        with tf.GradientTape() as tape:
            y_target_class_pred = self.predict_label(x_domain, training=False)
            target_lp_loss = self.loss(y_domain, y_target_class_pred)

        if mode == 0:
            self.test_lp_loss(target_lp_loss)
            self.test_lp_accuracy(y_domain, y_target_class_pred)

        if mode == 1:
            self.target_loss_1(target_lp_loss)
            self.target_accuracy_1(y_domain, y_target_class_pred)

        elif mode == 2:
            self.target_loss_2(target_lp_loss)
            self.target_accuracy_2(y_domain, y_target_class_pred)

        return

                          
    def return_latent_variables(self, x_domain):
    
        latent_variable = self.feature_extractor(x_domain, training=False)

        return latent_variable
        
    def log(self):
        message = log_format.format(
                self.test_lp_loss.result(),
                self.test_lp_accuracy.result()*100,
                self.test_dc_loss.result(),
                self.test_dc_accuracy.result()*100,
                self.test_target_lp_loss.result(),
                self.test_target_lp_accuracy.result()*100)
        
        with self.train_writer.as_default():
            tf.summary.scalar('label_prediction_loss', self.train_lp_loss.result(), step=self.lp_optimizer.iterations)
            
            tf.summary.scalar('label_prediction_accuracy', self.train_lp_accuracy.result(), step=self.lp_optimizer.iterations)
            tf.print('label_prediction_accuracy - Source: ',self.train_lp_accuracy.result())
            tf.summary.scalar('domain_classification_loss', self.train_dc_loss.result(), step=self.lp_optimizer.iterations)
            tf.summary.scalar('domain_classification_accuracy', self.train_dc_accuracy.result(), step=self.lp_optimizer.iterations)
            tf.print('domain_classification_accuracy:', self.train_dc_accuracy.result())
            
        with self.test_writer.as_default():
            tf.summary.scalar('label_prediction_loss', self.test_lp_loss.result(), step=self.lp_optimizer.iterations)
            tf.summary.scalar('label_prediction_accuracy', self.test_lp_accuracy.result(), step=self.lp_optimizer.iterations)
            tf.summary.scalar('domain_classification_loss', self.test_dc_loss.result(), step=self.lp_optimizer.iterations)
            tf.summary.scalar('domain_classification_accuracy', self.test_dc_accuracy.result(), step=self.lp_optimizer.iterations)
            tf.print('label_prediction_accuracy - Test:', self.test_lp_accuracy.result(), self.lp_optimizer.iterations)
        
        with self.target_writer.as_default():
            tf.summary.scalar('label_prediction_loss', self.test_target_lp_loss.result(), step=self.lp_optimizer.iterations)
            tf.summary.scalar('label_prediction_accuracy', self.test_target_lp_accuracy.result(), step=self.lp_optimizer.iterations)
            tf.print('label_prediction_accuracy - Target Test:', self.test_target_lp_accuracy.result())
            tf.print('label_prediction_accuracy - Target :', self.test_target_standard_lp_accuracy.result())
        self.reset_metrics('train')
        self.reset_metrics('test')

        return message

    def log_source_only(self):
        message = log_format.format(
            self.test_lp_loss.result(),
            self.test_lp_accuracy.result()*100,
            self.target_loss_1.result(),
            self.target_accuracy_1.result()*100,
            self.target_loss_2.result(),
            self.target_accuracy_2.result()*100)

        with self.train_writer.as_default():
            tf.summary.scalar('label_prediction_loss', self.train_lp_loss.result(), step=self.lp_optimizer.iterations)
            tf.summary.scalar('label_prediction_accuracy', self.train_lp_accuracy.result(), step=self.lp_optimizer.iterations)
            tf.print('label_prediction_accuracy - Train Only', self.train_lp_accuracy.result())
        with self.test_writer.as_default():
            tf.summary.scalar('label_prediction_loss', self.test_lp_loss.result(), step=self.lp_optimizer.iterations)
            tf.summary.scalar('label_prediction_accuracy', self.test_lp_accuracy.result(), step=self.lp_optimizer.iterations)
            tf.print('label_prediction_accuracy', self.test_lp_accuracy.result())
        with self.target_writer_1.as_default():
            tf.summary.scalar('label_prediction_loss', self.target_loss_1.result(), step=self.lp_optimizer.iterations)
            tf.summary.scalar('label_prediction_accuracy', self.target_accuracy_1.result(), step=self.lp_optimizer.iterations)

        with self.target_writer_2.as_default():
            tf.summary.scalar('label_prediction_loss', self.target_loss_2.result(), step=self.lp_optimizer.iterations)
            tf.summary.scalar('label_prediction_accuracy', self.target_accuracy_2.result(), step=self.lp_optimizer.iterations)

        self.reset_metrics('source_only')

        return message

    def create_logger(self, run_name, source_only, category):
        if os.path.isdir("../log/{}".format(run_name)):
            for i in range(99):
                if not os.path.isdir("../log/{}_{}".format(run_name, i)):
                    run_name = '{}_{}'.format(run_name, i)
                    break

        run_dir = "../log/{}".format(run_name)
        train_dir = "../log/{}/train".format(run_name)
        test_dir = "../log/{}/test".format(run_name)
        os.mkdir(run_dir)
        os.mkdir(train_dir)
        os.mkdir(test_dir)

        self.train_writer = tf.summary.create_file_writer(train_dir)
        self.test_writer = tf.summary.create_file_writer(test_dir)

        if source_only:
            target_dir_1 = "../log/{}/target_{}".format(run_name, category[0])
            target_dir_2 = "../log/{}/target_{}".format(run_name, category[1])
            os.mkdir(target_dir_1)
            os.mkdir(target_dir_2)
            self.target_writer_1 = tf.summary.create_file_writer(target_dir_1)
            self.target_writer_2 = tf.summary.create_file_writer(target_dir_2)

        else:
            target_dir = "../log/{}/target".format(run_name)
            os.mkdir(target_dir)
            self.target_writer = tf.summary.create_file_writer(target_dir)

        print("Log folder created as {}".format(run_dir))

        return

    def reset_metrics(self, target):

        if target == 'train':
            self.train_lp_loss.reset_states()
            self.train_lp_accuracy.reset_states()
            self.train_dc_loss.reset_states()
            self.train_dc_accuracy.reset_states()

        if target == 'test':
            self.test_lp_loss.reset_states()
            self.test_lp_accuracy.reset_states()
            self.test_dc_loss.reset_states()
            self.test_dc_accuracy.reset_states()
            self.test_target_lp_loss.reset_states()
            self.test_target_lp_accuracy.reset_states()	

        elif target == 'source_only':
            self.train_lp_loss.reset_states()
            self.train_lp_accuracy.reset_states()
            self.test_lp_loss.reset_states()
            self.test_lp_accuracy.reset_states()
            self.target_loss_1.reset_states()
            self.target_accuracy_1.reset_states()
            self.target_loss_2.reset_states()
            self.target_accuracy_2.reset_states()

        return

In [135]:
upsampling = Upsample(lr = [0.0001, 0.0001, 0.0001], run_name = '0.0001-0.0001-0.0001-huge-batchsize5')

Log folder created as ../log/0.0001-0.0001-0.0001-huge-batchsize5_23


In [0]:
import random

class Gen:
  def __init__(self,x,y,v, epochs = 10):
    self.xyv = iter(list(zip(x,y,v)))
    self.x = x
    self.y = y
    self.v = v
    self.epochs = epochs
    self.epoch = 1
  def __call__(self):
    while self.epoch < self.epochs:
      try: 
        yield next(self.xyv)
      except StopIteration:
        #break
        self.epoch += 1
        to_shuffle = list(zip(self.x,self.y,self.v))
        random.shuffle(to_shuffle)
        self.xyv = iter(to_shuffle)

def train_upsample_loading(model, dataset, x_class = None, y_class = None, x_domain = None, batch_size = None, epochs = 10, loading = True, SOURCE = True, TEST = True, TARGET = True):

        if batch_size == None:
            batch_size = dataset.batch_size
        if loading:
            source = dataset.source_paths
            size_source = len(source)
            print('Source size:', size_source)
          
            source = tf.data.Dataset.from_tensor_slices((source))
            source = list(source)
            if len(dataset.x_train) == 0: 
              dataset.loading_batches(batch = source, source = True, batch_size = size_source)
            source = tf.data.Dataset.from_generator(Gen(dataset.x_train, dataset.y_train, dataset.view_train, epochs = epochs),  (tf.float32, tf.int64, tf.int64))
            source = source.batch(batch_size)
        
        if TEST:
            test = dataset.test_paths
            size_test = len(test)
            print('Test size:', size_test)
            test = tf.data.Dataset.from_tensor_slices((test))
            test = test.shuffle(size_test, reshuffle_each_iteration=True)
            test = list(test)

            
        if TARGET:
            size = int(len(dataset.target_paths)/2)
            target = dataset.target_paths[:size]
            target_test = dataset.target_paths[size:]
            size_target = len(target)
            size_target_test = len(target_test)
            print('Target size:', size_target)
            print('Target_test size:', size_target_test)
            target_test = tf.data.Dataset.from_tensor_slices((target_test))
            target_test = target_test.shuffle(size_target_test, reshuffle_each_iteration=True)
            target_test = list(target_test)
            target = tf.data.Dataset.from_tensor_slices((target))
            target = list(target)
            if len(dataset.x_test) == 0:
              dataset.loading_batches(batch = target, target = True, batch_size = size_target)
            target = tf.data.Dataset.from_generator(Gen(dataset.x_test, dataset.y_test, dataset.view_test, epochs = epochs),  (tf.float32, tf.int64, tf.int64))
            target = target.batch(1) 

        if len(dataset.x_val) == 0:
          dataset.loading_batches(batch = test, test = True, batch_size = size_test)
        if len(dataset.x_target_test) == 0:
          dataset.loading_batches(batch = target_test, target_test = True, batch_size = size_target_test)
        x_target_test, y_target_test = dataset.x_target_test, dataset.y_target_test
        counter = 0
        epoch = 0
        print(datetime.datetime.now())
        print('Beginning of epoch:', epoch + 1)
        for (batch, batch_target) in zip(source,target):
            counter += 1
            if np.floor(counter * batch_size / size_source) > epoch:
                print(datetime.datetime.now())
                epoch = np.floor(counter * batch_size / size_source)
             
                tf.print('testing')
                

                
                model.test_source_only(dataset.x_train, dataset.y_train, dataset.view_train)
                model.test_only(dataset.x_val, dataset.y_val, mode = 0)
                model.test_target_test(x_target_test,y_target_test)
                model.test_target(dataset.x_test,dataset.y_test)
                model.log()
                    
                
                print('Beginning of epoch:', int(epoch + 1)) 
            x_class, y_class, x_domain, views_class, views_domain = batch[0], batch[1], batch_target[0], batch[2], batch_target[2]
            
               
            model.single_train(x_class, y_class, views_class, x_domain, views_domain, minimize_distance = False)
            
            #if (counter % 10) == 0:
            #    model.log()
        return

In [142]:
train_upsample_loading(upsampling, dataset, epochs =10)

Source size: 1056
Test size: 264
Target size: 165
Target_test size: 165
2020-02-06 18:08:07.363161
Beginning of epoch: 1
lp_loss 3.49946761
dc_loss 39.0641556
lp_loss 2.54713058
dc_loss 40.2730484
lp_loss 3.62689209
dc_loss 18.3539848
lp_loss 3.15368629
dc_loss 16.8620796
lp_loss 2.62905574
dc_loss 37.8601608
lp_loss 3.11778879
dc_loss 35.8976135
lp_loss 3.06702876
dc_loss 32.2674561
lp_loss 3.26675606
dc_loss 27.3756351
lp_loss 2.5071702
dc_loss 14.2174015
lp_loss 3.24115157
dc_loss 21.9157524
lp_loss 2.46361685
dc_loss 24.6340275
lp_loss 4.00801659
dc_loss 32.4018478
lp_loss 3.43130922
dc_loss 27.5224609
lp_loss 3.84101033
dc_loss 33.6922
lp_loss 3.62832689
dc_loss 47.8782082
lp_loss 3.47401285
dc_loss 34.5629158
lp_loss 3.14904118
dc_loss 42.1697
lp_loss 3.28364301
dc_loss 34.2444229
lp_loss 3.75248075
dc_loss 13.5921221
lp_loss 2.95999098
dc_loss 13.716116
lp_loss 3.07089233
dc_loss 16.4748516
lp_loss 3.2929554
dc_loss 54.5098457
lp_loss 2.58272076
dc_loss 20.4570713
lp_loss 3.0695

In [0]:
base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape = (48,52,3),classes = 11)
inputs = tf.keras.Input(shape=(48,52,3))
x = base_model.get_layer('block1_conv1')(inputs)
x = base_model.get_layer('block1_conv2')(x)
x = base_model.get_layer('block1_pool')(x)
x = base_model.get_layer('block2_conv1')(x)
y = Flatten()(x)
outputs = tf.keras.layers.Dense(11, activation=tf.nn.softmax)(y)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, decay=0.0002),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [144]:
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 48, 52, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 52, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 52, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 26, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 26, 128)       73856     
_________________________________________________________________
flatten_34 (Flatten)         (None, 79872)             0         
_________________________________________________________________
dense_34 (Dense)             (None, 11)                878

In [145]:
x1 = np.squeeze(tf.reshape(upsampling.return_latent_variables(dataset.x_train), (upsampling.return_latent_variables(dataset.x_train).shape[0], 48,52,3)))
x2 = np.squeeze(upsampling.return_latent_variables(dataset.x_target_test))
x_val = np.squeeze(tf.reshape(upsampling.return_latent_variables(dataset.x_val), (upsampling.return_latent_variables(dataset.x_val).shape[0], 48,52,3)))
y_val = dataset.y_val
y1 = dataset.y_train
y2 = dataset.y_target_test
y3 = dataset.y_test
#view1 = dataset.view_source
#view2 = dataset.view_target
#x = tf.concat([x1, x2], axis = 0)
#views = tf.concat([view1, view2], axis = 0)
model.fit(x1, y1, batch_size = 50, epochs = 20, verbose = True, validation_data = (x_val, y_val))

Train on 1056 samples, validate on 264 samples
Epoch 1/20
1056/1056 [==============================] - 1s 564us/sample - loss: 71.9950 - sparse_categorical_accuracy: 0.2254 - val_loss: 2.1741 - val_sparse_categorical_accuracy: 0.3485
Epoch 2/20
1056/1056 [==============================] - 0s 244us/sample - loss: 1.3781 - sparse_categorical_accuracy: 0.5322 - val_loss: 0.6968 - val_sparse_categorical_accuracy: 0.7841
Epoch 3/20
1056/1056 [==============================] - 0s 243us/sample - loss: 0.3352 - sparse_categorical_accuracy: 0.8816 - val_loss: 0.4009 - val_sparse_categorical_accuracy: 0.8636
Epoch 4/20
1056/1056 [==============================] - 0s 247us/sample - loss: 0.1621 - sparse_categorical_accuracy: 0.9479 - val_loss: 0.3696 - val_sparse_categorical_accuracy: 0.8561
Epoch 5/20
1056/1056 [==============================] - 0s 243us/sample - loss: 0.1112 - sparse_categorical_accuracy: 0.9688 - val_loss: 0.2758 - val_sparse_categorical_accuracy: 0.8977
Epoch 6/20
1056/1056 [

In [141]:
model.evaluate(np.squeeze(tf.reshape(upsampling.return_latent_variables(dataset.x_test), (upsampling.return_latent_variables(dataset.x_test).shape[0], 48,52,3))), y3)

165/165 [==============================] - 0s 142us/sample - loss: 9.9766 - sparse_categorical_accuracy: 0.1697


[9.976595572269325, 0.16969697]

In [43]:
model.get_layer('block2_conv1').output

<tf.Tensor 'block2_conv1/Identity:0' shape=(None, 24, 26, 128) dtype=float32>

In [47]:
from tf_explain.callbacks.grad_cam import GradCAMCallback

model1 = tf.keras.models.Model([model.inputs],[model.get_layer('block2_conv1').output, model.output])
callbacks = [
    GradCAMCallback(
      validation_data=(x_val,y_val),
        class_index=1,
        output_dir='../grad_cam'
    )]

model.fit(x1, y1, batch_size = 2, epochs = 1, verbose = True, callbacks = callbacks)


ValueError: ignored

In [39]:
from tf_explain.callbacks.occlusion_sensitivity import OcclusionSensitivityCallback

callbacks = [
    OcclusionSensitivityCallback(
        validation_data=(x_val, y_val),
        class_index=0,
        patch_size=2,
        output_dir='../occlusion',
    ),
]

model.fit(x1, y1, batch_size=2, epochs=2, callbacks=callbacks)

Train on 1056 samples
Epoch 1/2
1040/1056 [============================>.] - ETA: 0s - loss: 0.2500 - sparse_categorical_accuracy: 0.8942

IndexError: ignored

In [0]:
basic = BasicModel(lr = [0.1,0.001,0.001])

Log folder created as ../log/basic_model_13


In [0]:
dataset_crazy = Dataset(batch_size = 5, data_path_index = 1)
dataset_crazy.load_dataset_paths()

In [0]:
dataset_crazy.crossview_split(split_dict = split_dict['IXMAS']['allocentric'])

Training samples: 1056
Validation samples: 264
Test samples: 330


In [0]:
def train_basic(model_to_train, model_fe,  dataset, batch_size = None, epochs = 10):

        if batch_size == None:
            batch_size = dataset.batch_size
            
        source = dataset.source_paths  
        size_source = len(source)
        print('Source size:', size_source)
        source = tf.data.Dataset.from_tensor_slices((source))
        source = source.shuffle(size_source, reshuffle_each_iteration=True)
        source = source.repeat(epochs).batch(batch_size)
        
        test = dataset.test_paths
        size_test = len(test)
        print('Test size:', size_test)
        test = tf.data.Dataset.from_tensor_slices((test))
        test = test.shuffle(size_test, reshuffle_each_iteration=True)
        test = list(test)

        size_target = len(dataset.target_paths)
        target = dataset.target_paths
        print('Target size:', size_target)
        target = tf.data.Dataset.from_tensor_slices((target))
        target = list(target)

        dataset.loading_batches(batch = test, test = True, batch_size = size_test)
        dataset.loading_batches(batch = target, target_test = True, batch_size = size_target)
        x_target_test, y_target_test = dataset.x_target_test, dataset.y_target_test
        counter = 0
        epoch = 0
        print(datetime.datetime.now())
        print('Beginning of epoch:', epoch + 1)
        for batch in source:
            counter += 1
            if np.floor(counter * batch_size / size_source) > epoch:
                print(datetime.datetime.now())
                epoch = np.floor(counter * batch_size / size_source)
             
                tf.print('testing')

                #model_to_train.test_source_only(model_fe.return_latent_variables(dataset.x_val), dataset.y_val)
                model_to_train.test_target(model_fe.return_latent_variables(x_target_test),y_target_test)
                model_to_train.log()
                    
                
                print('Beginning of epoch:', int(epoch + 1)) 
            dataset.loading_batches(batch = batch, source = True, test = False, target = False, batch_size = batch_size)
            x_class, y_class, x_domain, views_class, views_domain = model_fe.return_latent_variables(dataset.x_train), dataset.y_train, dataset.x_test, dataset.view_train, dataset.view_test
            
               
            model_to_train.single_train(x_class, y_class)
            
            #if (counter % 10) == 0:
            #    model.log()
        return

In [0]:
train_basic(basic, upsampling,  dataset_crazy, epochs = 30)

Source size: 1056
Test size: 264
Target size: 330
2020-01-30 19:30:16.548076
Beginning of epoch: 1
lp_loss 1.16368818
lp_loss 0.545988441
lp_loss 0.920503438
lp_loss 1.94024372
lp_loss 0.508986354
lp_loss 0.604050279
lp_loss 0.235701561
lp_loss 0.205390617
lp_loss 0.899050713
lp_loss 0.347815812
lp_loss 2.99601722
lp_loss 0.77999258
lp_loss 4.49892902
lp_loss 1.20110118
lp_loss 0.893284142
lp_loss 1.04733431
lp_loss 1.48916423
lp_loss 3.09978056
lp_loss 0.951269507
lp_loss 2.57986856
lp_loss 0.287584662
lp_loss 1.10724473
lp_loss 0.757636368
lp_loss 0.376875252
lp_loss 0.258089453
lp_loss 3.13164759
lp_loss 0.245135739
lp_loss 1.39937532
lp_loss 3.98866272
lp_loss 1.40170515
lp_loss 0.608978152
lp_loss 1.38254619
lp_loss 0.694128215
lp_loss 0.0346269719
lp_loss 0.516841471
lp_loss 0.2021617
lp_loss 1.86776853
lp_loss 2.09439802
lp_loss 4.08180428
lp_loss 0.656950235
lp_loss 0.527894
lp_loss 1.23216677
lp_loss 3.55550504
lp_loss 1.00987244
lp_loss 0.224117905
lp_loss 0.607791781
lp_loss

KeyboardInterrupt: ignored